In [1]:
# Experiment 2
# Data cleaning and save the cleaned data into an Excel table, generate images, and store them in the specified folder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from openpyxl import load_workbook

In [2]:
# Define function to process a single file
abnormal_speed = 2.57
abnormal_distance = 48.35  # Distance anomaly; the participant should not move this far
settled_distance = 8.5  # The usual distance after teleportation in the initial experimental design
def process_file(file_path):
    # Define column names based on the observed structure
    column_names = ["Time", "ID", "Positionx", "Positionz", "Positiony", "Yaw", "Up", "Right", "Down", "Left"]

    # Read the CSV file without a header and assign column names
    data = pd.read_csv(file_path, header=None, names=column_names)

    # Adjust the time
    initial_time = data.loc[0, 'Time']
    data['Time'] = data['Time'] - initial_time

    # Remove parentheses from the Positionx and Positiony columns
    data['Positionx'] = data['Positionx'].astype(str).str.replace('(', '').str.replace(')', '')
    data['Positiony'] = data['Positiony'].astype(str).str.replace('(', '').str.replace(')', '')

    # Ensure the Positiony and Positionx columns contain numeric values after cleaning
    data['Positiony'] = pd.to_numeric(data['Positiony'], errors='coerce')
    data['Positionx'] = pd.to_numeric(data['Positionx'], errors='coerce')

    # Calculate the distance from the origin (0, 0) for each row and create a new 'Distance' column
    data['Distance'] = np.sqrt(data['Positionx']**2 + data['Positiony']**2)

    # Create an empty list to store indices of rows that need to be marked
    highlight_rows = []

    # Iterate through the Positiony column and filter rows that meet certain conditions
    for i in range(len(data) - 1):
        if -1.5 < data.loc[i, 'Positiony'] and abs(data.loc[i, 'Distance'] - data.loc[i + 1, 'Distance']) > 5:
            highlight_rows.append(i)
            highlight_rows.append(i + 1)

    # Create a new column to store labels
    data['Label'] = 0

    # Assign labels to the rows
    current_label = 1
    if highlight_rows:
        data.loc[:highlight_rows[0], 'Label'] = current_label
        for i in range(1, len(highlight_rows), 2):
            current_label += 1
            start_index = highlight_rows[i] + 1
            if i + 1 < len(highlight_rows):
                end_index = highlight_rows[i + 1]
                data.loc[start_index:end_index, 'Label'] = current_label
            else:
                data.loc[start_index:, 'Label'] = current_label

    # Replace the 0s in the Label column with the previous label
    for i in range(1, len(data)):
        if data.loc[i, 'Label'] == 0:
            data.loc[i, 'Label'] = data.loc[i + 1, 'Label']

    # Remove five rows of data (useless data during teleportation)

    # Create an empty list to store indices of rows that need to be cleared
    clear_rows = []

    # Iterate through the Positiony column and filter rows that meet certain conditions
    for i in range(len(data) - 1):
        if -1.5 < data.loc[i, 'Positiony'] and abs(data.loc[i, 'Distance'] - data.loc[i + 1, 'Distance']) > 5:
            for j in range(max(0, i-4), i+1):
                clear_rows.append(j)

    # Define the columns that need to be kept
    columns_to_keep = ['Time', 'ID']

    # Clear information but retain Time and ID columns
    for row_index in clear_rows:
        for col in data.columns:
            if col not in columns_to_keep:
                data.at[row_index, col] = np.nan  # This clears the content

    # Calculate the speed (positionx, positiony) at each time unit
    # Create a new Speed column
    data['Speed'] = np.nan

    # Iterate through the DataFrame and calculate the speed
    for i in range(1, len(data)):
        if pd.notna(data.at[i, 'Positionx']) and pd.notna(data.at[i, 'Positiony']) and \
           pd.notna(data.at[i-1, 'Positionx']) and pd.notna(data.at[i-1, 'Positiony']):
            # Calculate the distance
            dx = data.at[i, 'Positionx'] - data.at[i-1, 'Positionx']
            dy = data.at[i, 'Positiony'] - data.at[i-1, 'Positiony']
            distance = np.sqrt(dx**2 + dy**2)

            # Calculate the time difference
            dt = data.at[i, 'Time'] - data.at[i-1, 'Time']

            # Calculate the speed
            if dt != 0:
                speed = distance / dt
                data.at[i, 'Speed'] = speed

    # Filter out rows where speed is greater than 2.57 and distance is greater than 8.5, delete these rows and all previous rows in the same trajectory
    filtered_data = data[(data['Speed'] > abnormal_speed) & (data['Distance'] > settled_distance)]

    # Identify the labels corresponding to these rows
    labels_to_filter = filtered_data['Label'].unique()

    # Iterate through each label that needs processing
    for label in labels_to_filter:
        label_data = data[data['Label'] == label]
        # Find the last index to delete
        max_index_to_delete = filtered_data[filtered_data['Label'] == label].index.max()
        # Retain Time and ID columns, clear other columns
        for row_index in label_data[label_data.index <= max_index_to_delete].index:
            for col in data.columns:
                if col not in ['Time', 'ID']:
                    data.at[row_index, col] = np.nan  # Set to NaN

    # Reset the index
    data.reset_index(drop=True, inplace=True)

    # Filter out rows where distance is greater than 48.35 and delete these rows and all previous rows in the same trajectory
    
    # Identify the rows that meet the condition
    filtered_data = data[data['Distance'] > abnormal_distance]

    # Identify labels corresponding to these rows
    labels_to_filter = filtered_data['Label'].unique()

    # Iterate through each label that needs processing
    for label in labels_to_filter:
        label_data = data[data['Label'] == label]
        # Find the last index to delete
        max_index_to_delete = filtered_data[filtered_data['Label'] == label].index.max()
        # Retain Time and ID columns, clear other columns
        for row_index in label_data[label_data.index <= max_index_to_delete].index:
            for col in data.columns:
                if col not in ['Time', 'ID']:
                    data.at[row_index, col] = np.nan  # Set to NaN

    # Reset the index
    data.reset_index(drop=True, inplace=True)

    # Filter out incomplete trajectories, i.e., those without distances falling within 1.8 and 7
    data['Distance'] = pd.to_numeric(data['Distance'], errors='coerce')
    # Step 1: Find the minimum and maximum distance for each trajectory
    trajectory_stats = data.groupby('Label')['Distance'].agg(['min', 'max']).reset_index()
    # Step 2: Identify incomplete trajectories
    incomplete_trajectories = trajectory_stats[
        ~((trajectory_stats['min'] < 1.8) & (trajectory_stats['max'] > 7))
    ]['Label']

    # Step 3: Set data to NaN for incomplete trajectories except for ID and Time
    for label in incomplete_trajectories:
        data.loc[data['Label'] == label, data.columns.difference(['Time', 'ID'])] = np.nan

    # Remove stationary points during rest periods (considering the situation of immobility due to crowding)
    speed_threshold = 0.1
    distance_threshold = 2.7
    consecutive_count_threshold = 20

    # Store labels and indices that meet the condition
    labels_to_remove = set()

    # Iterate through each label
    for label in data['Label'].unique():
        if pd.notna(label):
            label_data = data[data['Label'] == label]
            consecutive_count = 0
            for index, row in label_data.iterrows():
                if pd.notna(row['Speed']) and row['Speed'] < speed_threshold and row['Distance'] > distance_threshold:
                    consecutive_count += 1
                    if consecutive_count > consecutive_count_threshold:
                        labels_to_remove.add(label)
                        break
                else:
                    consecutive_count = 0

    deleted_rows = pd.DataFrame(columns=data.columns)

    for label in labels_to_remove:
        label_data = data[data['Label'] == label]
        max_index_to_delete = label_data.index.max()
        deleted_rows = pd.concat([deleted_rows, label_data[label_data.index <= max_index_to_delete]])
        for row_index in label_data.index:
            if row_index <= max_index_to_delete:
                for col in data.columns:
                    if col not in ['Time', 'ID']:
                        data.at[row_index, col] = np.nan

    data.reset_index(drop=True, inplace=True)

    # Remove the points of idleness before the experiment ends
    speed_threshold = 0.1
    last_valid_index = data[data['Speed'] >= speed_threshold].index.max()
    if not np.isnan(last_valid_index):
        for row_index in range(last_valid_index + 1, len(data)):
            for col in data.columns:
                if col not in ['Time', 'ID']:
                    data.at[row_index, col] = np.nan

    data.reset_index(drop=True, inplace=True)

    data['AngleChange'] = np.nan

    # Function to calculate the change in angle
    def calculate_angle_change(x0, y0, x1, y1, x2, y2):
        v1 = np.array([x1 - x0, y1 - y0])
        v2 = np.array([x2 - x1, y2 - y1])
        dot_product = np.dot(v1, v2)
        norm_v1 = np.linalg.norm(v1)
        norm_v2 = np.linalg.norm(v2)
        if norm_v1 == 0 or norm_v2 == 0:
            return 0
        cos_theta = dot_product / (norm_v1 * norm_v2)
        cos_theta = np.clip(cos_theta, -1.0, 1.0)
        angle = np.arccos(cos_theta) * 180 / np.pi
        return angle

    # Iterate and calculate the angle change for each trajectory point
    for i in range(1, len(data) - 1):
        if pd.notna(data.at[i-1, 'Positionx']) and pd.notna(data.at[i-1, 'Positiony']) and \
           pd.notna(data.at[i, 'Positionx']) and pd.notna(data.at[i, 'Positiony']) and \
           pd.notna(data.at[i+1, 'Positionx']) and pd.notna(data.at[i+1, 'Positiony']):
            angle_change = calculate_angle_change(
                data.at[i-1, 'Positionx'], data.at[i-1, 'Positiony'],
                data.at[i, 'Positionx'], data.at[i, 'Positiony'],
                data.at[i+1, 'Positionx'], data.at[i+1, 'Positiony']
            )
            data.at[i, 'AngleChange'] = angle_change

    # Ensure the first row in each trajectory has NaN in the Speed column (no inertia)
    unique_labels = data['Label'].dropna().unique()
    for label in unique_labels:
        first_index = data[data['Label'] == label].index.min()
        data.at[first_index, 'Speed'] = np.nan
    
    # Calculate SpeedChange
    data['SpeedChange'] = np.nan  # Create a new SpeedChange column and initialize it to NaN

    # Iterate and calculate SpeedChange
    for i in range(1, len(data)):
        if pd.notna(data.at[i, 'Speed']) and pd.notna(data.at[i-1, 'Speed']):
            data.at[i, 'SpeedChange'] = data.at[i, 'Speed'] - data.at[i-1, 'Speed']

    return data  # Return the data for further use


In [3]:
def plot_traj(data, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    data_with_labels = data.dropna(subset=['Label'])
    label_times = data_with_labels.groupby('Label')['Time'].agg(['min', 'max']).reset_index()

    for label in label_times['Label'].unique():
        track_data = data_with_labels[data_with_labels['Label'] == label]
        if track_data.empty:
            continue
        track_data = track_data.dropna(subset=['Positionx', 'Positiony'])
        start_time = label_times[label_times['Label'] == label]['min'].values[0]
        end_time = label_times[label_times['Label'] == label]['max'].values[0]
        participant_id = track_data['ID'].values[0]
        fig, ax = plt.subplots()
        ax.plot(track_data['Positionx'], track_data['Positiony'], marker='o')
        ax.set_title(f'ID {participant_id} - Track {label} (Start: {start_time}, End: {end_time})')
        ax.set_xlabel('Positionx')
        ax.set_ylabel('Positiony')
        ax.grid(True)
        # Save the image to the specified folder
        fig.savefig(os.path.join(output_folder, f'track_{participant_id}_label_{label}.png'))
        plt.close(fig)  # Close the current figure to free memory


In [4]:
# Define folder path
folder_path = '/Users/yangfanzhou/Desktop/1.8/Experiment 2 data'
image_output_folder = '/Users/yangfanzhou/Desktop/1.8/Experiment2_AllTrajectories'

csv_file = glob.glob(os.path.join(folder_path, '*.csv'))


In [5]:
process_dfs = []

# Loop through each file in csv_file
for file in csv_file:
    # Process each file and store the resulting DataFrame
    process_df = process_file(file)
    process_dfs.append(process_df)
    
    # Generate trajectory plots for each processed DataFrame
    plot_traj(process_df, image_output_folder)

# Combine all processed DataFrames into one DataFrame
combined_df = pd.concat(process_dfs, ignore_index=True)

# Display the combined DataFrame (optional, depending on your environment)
combined_df

# Save the combined DataFrame to the specified Excel file
output_excel_path = '/Users/yangfanzhou/Desktop/1.8/ResultWholeDistance/Experiment2_Data.xlsx'
combined_df.to_excel(output_excel_path, index=False)


/var/folders/8t/9k64w8hn15ng50h2zwfs07tr0000gn/T/ipykernel_45401/297892584.py:194: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  deleted_rows = pd.concat([deleted_rows, label_data[label_data.index <= max_index_to_delete]])
/var/folders/8t/9k64w8hn15ng50h2zwfs07tr0000gn/T/ipykernel_45401/297892584.py:194: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  deleted_rows = pd.concat([deleted_rows, label_data[label_data.index <= max_index_to_delete]])
/var/folders/8t/9k64w8hn15ng50h2zwfs07tr0000gn/T/ipyke